[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/trainNNmodel.ipynb)

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers

from keras.models import load_model


Using TensorFlow backend.


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  model_XGB.sav  user_test_n.csv
model_NN.h5    output.csv     user_train_n.csv


In [0]:
s_data = 'user_train_n.csv'
s_col_tgt = 'transactionRevenue_sum'
l_col_drop = [s_col_tgt]
filename = 'model_NN.h5'

In [21]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

CPU times: user 8.73 s, sys: 379 ms, total: 9.11 s
Wall time: 12.3 s


<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


In [22]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 64.8 ms, sys: 186 ms, total: 251 ms
Wall time: 250 ms


In [0]:
def build_model(N_col):
    model = models.Sequential()
    model.add(layers.Dense(120, activation='relu', input_shape=(N_col,)))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1, activation='relu'))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [25]:
%%time
epochs = 100
batch_size = df_x.shape[0] // 4
validation_split = 0.1

model = build_model(df_x.shape[1])

history = model.fit(df_x,
                    df_y,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split = validation_split,
                    verbose=1)

Train on 642750 samples, validate on 71417 samples
Epoch 1/100
642750/642750 [==============================] - 3s 4us/step - loss: 8.2829 - mean_absolute_error: 0.5196 - val_loss: 8.3734 - val_mean_absolute_error: 0.4140
Epoch 2/100
642750/642750 [==============================] - 2s 3us/step - loss: 8.1599 - mean_absolute_error: 0.4938 - val_loss: 8.2576 - val_mean_absolute_error: 0.4942
Epoch 3/100
642750/642750 [==============================] - 2s 3us/step - loss: 8.0708 - mean_absolute_error: 0.5173 - val_loss: 8.2045 - val_mean_absolute_error: 0.4177
Epoch 4/100
642750/642750 [==============================] - 2s 3us/step - loss: 8.0098 - mean_absolute_error: 0.5018 - val_loss: 8.1279 - val_mean_absolute_error: 0.4401
Epoch 5/100
642750/642750 [==============================] - 2s 3us/step - loss: 7.9485 - mean_absolute_error: 0.5026 - val_loss: 8.0686 - val_mean_absolute_error: 0.4347
Epoch 6/100
642750/642750 [==============================] - 2s 3us/step - loss: 7.8918 - mean

In [26]:
history.history

{'loss': [8.282924135805336,
  8.15993939528953,
  8.07081938099722,
  8.009826560936286,
  7.948502566401757,
  7.8917853548731705,
  7.837941039087243,
  7.782774447575009,
  7.728325665774024,
  7.68427846422125,
  7.635262543667527,
  7.567878878539659,
  7.528131185292734,
  7.486337178560047,
  7.417865435019767,
  7.384787861186398,
  7.332837472854816,
  7.278771216798691,
  7.244469954843421,
  7.19323498632638,
  7.15998997009037,
  7.1216815596958485,
  7.07258150765498,
  7.033943174588453,
  6.982432563162276,
  6.970278197554474,
  6.911163377212115,
  6.896459172091935,
  6.854881460428701,
  6.8176668897923305,
  6.820145487707072,
  6.744109689121996,
  6.764890560971397,
  6.699770775195181,
  6.652452368288548,
  6.617228736953854,
  6.636087570450826,
  6.631894564178657,
  6.568010095091842,
  6.571964167481033,
  6.5129819595147,
  6.504089489699801,
  6.493190653852524,
  6.483126294317435,
  6.430329718265215,
  6.4307978137731086,
  6.43556658502072,
  6.416945

In [27]:
%%time
model.save(filename)

#files.download(filename)

CPU times: user 44.1 ms, sys: 7.98 ms, total: 52.1 ms
Wall time: 525 ms
